In [1]:
import pandas as pd
import os
from sklearn import preprocessing  # pip install sklearn
from collections import deque
import random
import numpy as np

# Longitud de la secuencia precedente para recolectar para la RNN
SEQ_LEN = 60

# Cuánto tiempo en el futuro estamos tratando de predecir
FUTURE_PERIOD_PREDICT = 8

# El ratio que queremos predecir
RATIO_TO_PREDICT = "DOGE"

# Función para clasificar si el valor futuro es mayor que el valor actual
def classify(current, future):
    if float(future) > float(current):
        return 1
    else:
        return 0

# Función para preprocesar el DataFrame
def preprocess_df(df):
    df = df.drop(columns=["future"])  # ya no la necesitamos.

    for col in df.columns:  # recorrer todas las columnas
        if col != "target":  # normalizar todas excepto la columna objetivo
            df[col] = df[col].pct_change()  # cambio porcentual "normaliza" las diferentes monedas (cada criptomoneda tiene valores muy diferentes, estamos más interesados en los movimientos de otras monedas)
            df.dropna(inplace=True)  # eliminar los valores nulos creados por pct_change
            df[col] = preprocessing.scale(df[col].values)  # escalar entre 0 y 1.

    df.dropna(inplace=True)  # limpieza adicional... por si acaso. Esos molestos NaNs tienden a colarse.
    sequential_data = []  # esta es una lista que CONTENDRÁ las secuencias
    prev_days = deque(maxlen=SEQ_LEN)  # Estas serán nuestras secuencias reales. Se hacen con deque, que mantiene la longitud máxima eliminando los valores más antiguos a medida que entran nuevos.

    for i in df.values:  # iterar sobre los valores
        prev_days.append([n for n in i[:-1]])  # almacenar todo excepto el objetivo
        if len(prev_days) == SEQ_LEN:  # asegurarse de tener 60 secuencias
            sequential_data.append([np.array(prev_days), i[-1]])  # agregar esas secuencias!

    random.shuffle(sequential_data)  # mezclar por buena medida

# Uso de una cadena sin procesar para evitar errores de escape de unicode
file_path = r"C:\Users\CRISTIAN CHAVEZ\Documents\GitHub\AI_Project_Deep-Learning-Tensorflow-Keras\Modelo Para Predecir Criptomonedas\criptodata\DOGE.xlsx"

# Leer el archivo inicial
df = pd.read_excel(file_path, names=['time', 'open', 'high', 'low', 'close', 'volume', 'usdVolume'])

# DataFrame principal vacío para almacenar todos los datos
main_df = pd.DataFrame()

# Las 4 criptomonedas que queremos considerar
ratios = ["BTC", "DOGE", "ETH", "QTU"]

# Ruta base donde se encuentran los archivos de datos
base_path = r"C:\Users\CRISTIAN CHAVEZ\Documents\GitHub\AI_Project_Deep-Learning-Tensorflow-Keras\Modelo Para Predecir Criptomonedas\criptodata"

# Iterar sobre cada ratio
for ratio in ratios:
    print(ratio)
    
    # Obtener la ruta completa al archivo
    dataset = os.path.join(base_path, f"{ratio}.xlsx")
    
    # Leer el archivo específico
    df = pd.read_excel(dataset, names=['time', 'open', 'high', 'low', 'close', 'volume', 'usdVolume'])
    
    # Renombrar columnas de 'close' y 'volume' para incluir el ticker
    df.rename(columns={"close": f"{ratio}_close", "volume": f"{ratio}_volume"}, inplace=True)
    
    # Establecer 'time' como índice para poder unirlos por este tiempo compartido
    df.set_index("time", inplace=True)
    
    # Ignorar otras columnas además de 'close' y 'volume'
    df = df[[f"{ratio}_close", f"{ratio}_volume"]]
    
    # Si el DataFrame principal está vacío, simplemente asignar el DataFrame actual
    if len(main_df) == 0:
        main_df = df
    else:
        # De lo contrario, unir estos datos con el DataFrame principal
        main_df = main_df.join(df)

# Si hay huecos en los datos, usar los valores conocidos anteriormente
main_df.fillna(method="ffill", inplace=True)
# Eliminar filas con valores nulos
main_df.dropna(inplace=True)

# Imprimir las primeras filas del DataFrame resultante para verificar los datos
print(main_df.head())

# Crear una nueva columna 'future' que es el valor de cierre futuro desplazado por 'FUTURE_PERIOD_PREDICT'
main_df['future'] = main_df[f'{RATIO_TO_PREDICT}_close'].shift(-FUTURE_PERIOD_PREDICT)

# Crear una columna 'target' que clasifica si el valor futuro es mayor que el valor actual
main_df['target'] = list(map(classify, main_df[f'{RATIO_TO_PREDICT}_close'], main_df['future']))

# Imprimir las primeras filas del DataFrame resultante para verificar las nuevas columnas
print(main_df[[f"{RATIO_TO_PREDICT}_close", "future"]].head())
#print(main_df.head())  # Imprimir las primeras filas del DataFrame resultante para verificar la estructura y los datos cargados correctamente.


BTC
DOGE
ETH
QTU
            BTC_close  BTC_volume  DOGE_close  DOGE_volume  ETH_close  \
time                                                                    
1715356800    60793.4      15.101    0.144841     169877.0    2914.04   
1715356860    60729.3       6.395    0.144513     245666.0    2907.25   
1715356920    60744.5       7.272    0.144437     417310.0    2906.65   
1715356980    60713.9       4.568    0.144207     311595.0    2904.65   
1715357040    60683.8      18.586    0.144067     153653.0    2901.04   

            ETH_volume  QTU_close  QTU_volume  
time                                           
1715356800        45.7      3.629      1283.8  
1715356860        23.8      3.620        44.7  
1715356920       997.9      3.611        20.8  
1715356980        82.0      3.606        46.6  
1715357040        96.0      3.609        32.8  
            DOGE_close    future
time                            
1715356800    0.144841  0.144547
1715356860    0.144513  0.144578
171

C:\Users\CRISTIAN CHAVEZ\AppData\Local\Temp\ipykernel_33564\3084470778.py:87: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  main_df.fillna(method="ffill", inplace=True)


In [2]:
# Obtener una lista ordenada de los valores del índice (los tiempos)
times = sorted(main_df.index.values)

# Calcular el punto de corte para el último 5% de los tiempos
last_5pct = sorted(main_df.index.values)[-int(0.05 * len(times))]

# Imprimir el valor del tiempo que representa el inicio del último 5% del conjunto de datos
print(last_5pct)


1715438880


In [3]:
# Crear el DataFrame de validación con los datos donde el índice está en el último 5% de los tiempos
validation_main_df = main_df[(main_df.index >= last_5pct)]  # crear los datos de validación donde el índice está en el último 5%
# Actualizar el DataFrame principal para que contenga solo los datos hasta el último 5%
main_df = main_df[(main_df.index < last_5pct)]  # ahora el DataFrame principal contiene todos los datos hasta el último 5%

# Preprocesar los datos principales
preprocess_df(main_df)  # llamar a la función de preprocesamiento para los datos principales
#train_x, train_y = preprocess_df(main_df) 
#validation_x, validation_y = preprocess_df(validation_main_df)